In [1]:
from ioMicro import *

In [2]:
asm = analysis_smFISH(data_folders = [r'Z:\DCBBL1_3_2_2023\MERFISH',r'\\192.168.0.3\bbfishdc10_2\DCBBL1_3_2_2023',r'Z:\DCBBL1_3_2_2023\Controls'],
                 save_folder =r'Z:\DCBBL1_3_2_2023\MERFISH_Analysis',
                 H0folder=  r'Z:\DCBBL1_3_2_2023\Controls\H0*',exclude_H0=False)

Found files:1295
Found hybe folders:72


In [3]:
ifov=70
asm.set_fov(ifov,set_='_set1')
asm.fov =os.path.basename(asm.fl_bk)

MERFISH_flds = [fld for fld in asm.Qfolders if asm.set_ in os.path.basename(fld) if 'MERFISH' in fld]
ctrl_flds  = [fld for fld in asm.Qfolders if asm.set_ in os.path.basename(fld) if ('ptb' in os.path.basename(fld).lower()) or ('H0' in os.path.basename(fld))]
all_flds = np.array(ctrl_flds+MERFISH_flds)

def get_iH(fld): return int(os.path.basename(fld).split('_')[0][1:])
all_flds = all_flds[np.argsort([get_iH(fld) for fld in all_flds])]
all_flds

array(['Z:\\DCBBL1_3_2_2023\\Controls\\H0_set1',
       'Z:\\DCBBL1_3_2_2023\\Controls\\H1_Igfbp_Aldh1l1_Ptbp1_set1',
       'Z:\\DCBBL1_3_2_2023\\MERFISH\\H1_MER_set1',
       'Z:\\DCBBL1_3_2_2023\\MERFISH\\H2_MER_set1',
       'Z:\\DCBBL1_3_2_2023\\MERFISH\\H3_MER_set1',
       'Z:\\DCBBL1_3_2_2023\\MERFISH\\H4_MER_set1',
       'Z:\\DCBBL1_3_2_2023\\MERFISH\\H5_MER_set1',
       'Z:\\DCBBL1_3_2_2023\\MERFISH\\H6_MER_set1',
       'Z:\\DCBBL1_3_2_2023\\MERFISH\\H7_MER_set1',
       'Z:\\DCBBL1_3_2_2023\\MERFISH\\H8_MER_set1',
       'Z:\\DCBBL1_3_2_2023\\MERFISH\\H9_MER_set1',
       'Z:\\DCBBL1_3_2_2023\\MERFISH\\H10_MER_set1',
       'Z:\\DCBBL1_3_2_2023\\MERFISH\\H11_MER_set1',
       'Z:\\DCBBL1_3_2_2023\\MERFISH\\H12_MER_set1',
       'Z:\\DCBBL1_3_2_2023\\MERFISH\\H13_MER_set1',
       'Z:\\DCBBL1_3_2_2023\\MERFISH\\H14_MER_set1'], dtype='<U55')

In [ ]:
asm.save

In [5]:
def compute_fits(save_folder,fov,all_flds,redo=False):
    for fld in tqdm(all_flds):
        im_ = read_im(fld+os.sep+asm.fov)
        ncols = len(im_)
        for icol in range(ncols-1):
            #icol=2
            save_fl = asm.save_folder+os.sep+asm.fov.split('.')[0]+'--'+tag+'--col'+str(icol)+'__Xhfits.npy'
            if not os.path.exists(save_fl) or redo:
                im__ = np.array(im_[icol],dtype=np.float32)
                im_n = norm_slice(im__,s=30)
                Xh = get_local_max(im_n,500,im_raw=im__,dic_psf=None,delta=1,delta_fit=3,dbscan=True,
                      return_centers=False,mins=None,sigmaZ=1,sigmaXY=1.5)
                tag = os.path.basename(fld)
                np.savez_compressed(save_fl,Xh=Xh)

100%|█████████████████████████████████████████████████████████████████████████████████| 16/16 [44:18<00:00, 166.16s/it]


In [9]:
def compute_drift(save_folder,fov,all_flds,redo=False):
    drift_fl = save_folder+os.sep+'drift_'+fov.split('.')[0]+'.pkl'
    if not os.path.exists(drift_fl) or redo:
        ims = [read_im(fld+os.sep+fov) for fld in all_flds]
        ncols,sz,sx,sy = ims[0].shape
        im_ref = np.array(ims[len(ims)//2][-1],dtype=np.float32)
        drifts = [get_txyz(im[-1],im_ref,sz_norm=20, sz=500) for im in tqdm(ims)]
        drift_fl = asm.save_folder+os.sep+'drift_'+asm.fov.split('.')[0]+'.pkl'
        pickle.dump([drifts,all_flds,asm.fov],open(drift_fl,'wb'))

In [10]:
compute_drift(asm.save_folder,asm.fov,all_flds,redo=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [11:19<00:00, 42.49s/it]
